# Python to Javascript construct mapping tests

Test that expressions evaluated for widgets in the Python context
do something similar to what they would do in Javascript context

In [1]:
import jp_proxy_widget

from jp_proxy_widget import notebook_test_helpers

validators = notebook_test_helpers.ValidationSuite()

In [2]:
class Callback:
    
    def __init__(self):
        self.called = False
        
    def __call__(self, argument=True):
        self.called = argument

# Chain of attributes

In [3]:
ca_widget = jp_proxy_widget.JSProxyWidget()
ca_callback = Callback()
ca_widget.debugging_display()

In [4]:
ca_widget.js_init("""
    element.html("chain of attributes test widget");
    element.A = {};
    element.A.B = {};
    element.A.B.C = (function() { return callback(); });
    
    // emulate this:
    //element.A.B.C()
""", callback=ca_callback)

In [5]:
dummy = ca_widget.element.A.B.C()

In [6]:
def validate_ca():
    assert ca_callback.called is True
    print("Chain of attributes ok.")
    
#validate_ca()
validators.add_validation(ca_widget, validate_ca)

# Chain of methods

In [7]:
cm_widget = jp_proxy_widget.JSProxyWidget()
cm_callback = Callback()

cm_widget.js_init("""
    element.html("chain of methods test widget");
    element.A = (
        function() {
            return {
                B: (function() { callback() })
            }
        }
    );
    // emulate this:
    //element.A().B()
""", callback=cm_callback)

a = cm_widget.element.A()

ab = cm_widget.element.A().B()

cm_widget.debugging_display()

In [8]:
def validate_cm():
    assert cm_callback.called is True
    print("Chain of methods ok.")
    
#validate_cm()
validators.add_validation(cm_widget, validate_cm)
a, ab

(FragileReference(140209362663800), FragileReference(140209362664136))

In [9]:
try:
    a()
except jp_proxy_widget.StaleFragileJavascriptReference:
    print ("As expected: you can't use a fragile javascript reference outside a chained expression.")
else:
    raise SystemError("Oops: We should have gotten a StaleFragileJavascriptReference raised here.")

As expected: you can't use a fragile javascript reference outside a chained expression.


# Chained attribute/method

In [10]:
cam_widget = jp_proxy_widget.JSProxyWidget()
cam_callback = Callback()

cam_widget.js_init("""
    element.html("chain of object with a function attribute");
    element.A = {
        B: function(f) { f("expected value") }
    };
    element
    // emulate this:
    //element.A.B(cam_callback)
""", cam_callback=cam_callback)

#ab = cam_widget.element.A().B()

cam_widget.debugging_display()

In [11]:
cam_widget.element.A.B(cam_callback)

FragileReference(140209362726360)

In [12]:
def validate_cam():
    assert cam_callback.called == "expected value"
    print("Chain of attribute/function ok.")
    
#validate_cam()
validators.add_validation(cam_widget, validate_cam)

In [13]:
delay_ms = 1000
validators.run_all_in_widget(delay_ms=delay_ms)

sleeping in kernel interface...
initializing validator widget.
